### Support Multiple language for post analysis

In [3]:
!pip install streamlit
!pip install pyngrok
!pip install praw
!pip install googletrans

In [ ]:
%%writefile app06.py
import streamlit as st
import joblib
import pandas as pd
import praw
from collections import Counter
from googletrans import Translator

# Load the saved model and vectorizer
model = joblib.load('LRmodel.pkl')
vectorizer = joblib.load('LRvectorizer.pkl')

# Initialize Reddit API client and Translator
reddit = praw.Reddit(client_id='<REDDIT_CLIENT_ID>',
                     client_secret='<REDDIT_CLIENT_SECRET>',
                     user_agent='Mental Health')
translator = Translator()

# Function to fetch user posts
def fetch_user_posts(username):
    try:
        user = reddit.redditor(username)
        posts = [post.title + " " + post.selftext for post in user.submissions.new(limit=20)]
        return posts
    except Exception as e:
        st.write(f"Error fetching posts: {e}")
        return []

# Define the Streamlit app
def run_app():
    st.title('Mental Health Classifier with Reddit Username and Translation')

    # Reddit username input
    username = st.text_input("Enter Reddit username:")

    # Button to fetch posts and classify
    if st.button("Analyze"):
        if username.strip() == "":
            st.write("Please enter a Reddit username.")
        else:
            # Fetch posts
            posts = fetch_user_posts(username)
            if not posts:
                st.write("No posts found or unable to fetch posts for this user.")
            else:
                # Display the original posts
                st.write("Some of the user's recent posts (original language):")
                st.write(posts[:3])  # Display a few posts for review

                # Classify each post separately
                predictions = []
                for post in posts:
                    # Translate post to English if needed
                    try:
                        translated_text = translator.translate(post, dest='en').text
                    except Exception as e:
                        st.write(f"Translation error: {e}")
                        translated_text = post  # Fallback to original text

                    # Vectorize the translated text and classify
                    input_vectorized = vectorizer.transform([translated_text])
                    prediction = model.predict(input_vectorized)
                    predictions.append(prediction[0])

                # Count the most common mental health issue
                issue_counts = Counter(predictions)
                top_issue, top_count = issue_counts.most_common(1)[0]
                top_percentage = (top_count / len(predictions)) * 100

                # Output the result
                st.write(f"The most frequently detected mental health concern is: {top_issue} appearing in {top_percentage:.2f}% of posts.")

                # Display all issues and their counts for reference
                st.write("Mental health issue distribution across posts:")
                issue_distribution = pd.DataFrame(issue_counts.items(), columns=['Mental Health Issue', 'Count'])
                st.write(issue_distribution)

# Run the app
if __name__ == '__main__':
    run_app()


Writing app06.py


In [ ]:
# Import ngrok
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("<NGROK_AUTH_TOKEN>") # Replace YOUR_AUTHTOKEN with your actual authtoken

# Kill any existing ngrok processes
ngrok.kill()

# Start Streamlit with nohup
!nohup streamlit run app06.py &

# Create a public URL with ngrok to access the app
public_url = ngrok.connect(addr='8501')
print(f"Public URL: {public_url}")

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://d506-34-80-55-190.ngrok-free.app" -> "http://localhost:8501"


In [6]:
ngrok.kill()

In [7]:
!jupyter nbconvert --to html Test_06.ipynb

[NbConvertApp] Converting notebook Test_06.ipynb to html
[NbConvertApp] Writing 295542 bytes to Test_06.html
